In [3]:
import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import mnist
from keras.models import Model,Sequential
from keras.layers import Input,Dense
from keras.utils import np_utils

In [ ]:
(previsores_treinamento,classe_treinamento), (previsores_teste,classe_teste) = mnist.load_data()

In [5]:
previsores_treinamento = previsores_treinamento.astype('float32')/255
previsores_teste = previsores_teste.astype('float32')/255
previsores_treinamento = previsores_treinamento.reshape(len(previsores_treinamento), np.prod(previsores_treinamento.shape[1:]))
previsores_teste = previsores_teste.reshape(len(previsores_teste), np.prod(previsores_teste.shape[1:]))

In [28]:
def reduzirInformacao(tamanho, previsores):
    numeroNeuroniosEntrada = previsores[0].shape[1]
    numeroNeuroniosSaida = previsores[0].shape[1]
    numeroNeuroniosCamadaOculta = tamanho
    autoencoder = Sequential()
    autoencoder.add(Dense(units= numeroNeuroniosCamadaOculta, activation='relu', input_dim = numeroNeuroniosEntrada))
    autoencoder.add(Dense(units= numeroNeuroniosSaida, activation = 'sigmoid'))
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    autoencoder.fit(previsores[0],previsores[0], epochs=50, batch_size=256,validation_data=(previsores[1],previsores[1]))
    
    dimensaoOriginal = Input(shape=(numeroNeuroniosEntrada,))
    camada_encoder = autoencoder.layers[0]
    encoder = Model(dimensaoOriginal, camada_encoder(dimensaoOriginal))

    return encoder


In [ ]:
encoder = reduzirInformacao(32, (previsores_treinamento,previsores_teste))
previsores_treinamento_codificados = encoder.predict(previsores_treinamento)
previsores_teste_codificados = encoder.predict(previsores_teste)

In [9]:
classe_dummy_treinamento = np_utils.to_categorical(classe_treinamento)
classe_dummy_teste = np_utils.to_categorical(classe_teste)

In [15]:
def criarRedeNeuralClassificacao(previsores,classe):
    numeroNeuroniosEntrada = previsores[0].shape[1]
    numeroNeuroniosSaida = classe[0].shape[1]
    numeroNeuroniosCamadaOculta = round((numeroNeuroniosEntrada+numeroNeuroniosSaida)/2)
    c = Sequential()
    c.add(Dense(units= numeroNeuroniosCamadaOculta, activation='relu', input_dim = numeroNeuroniosEntrada))
    c.add(Dense(units= numeroNeuroniosCamadaOculta, activation= 'relu'))
    c.add(Dense(units= numeroNeuroniosSaida, activation= 'softmax'))
    c.compile(optimizer = 'adam', loss= 'categorical_crossentropy', metrics = ['accuracy'])
    c.fit(previsores[0], classe[0], batch_size = 256, epochs = 100, validation_data = (previsores[1], classe[1]))
    return c

Accuracy: 98%

In [ ]:
c1 = criarRedeNeuralClassificacao((previsores_treinamento, previsores_teste),(classe_dummy_treinamento, classe_dummy_teste)) #Base normal de 32x32

Accuracy: 95%

In [ ]:
c2 = criarRedeNeuralClassificacao((previsores_treinamento_codificados, previsores_teste_codificados),(classe_dummy_treinamento, classe_dummy_teste)) #Base normal de 8x4